In [1]:
import pandas as pd
import numpy as np
import import_ipynb
import dividend_detective_scrape

import yfinance as yf

import datetime as dt

from datetime import datetime

importing Jupyter notebook from dividend_detective_scrape.ipynb


In [2]:
%%time
div_stocks = dividend_detective_scrape.get_data()

CPU times: user 29.9 ms, sys: 17.2 ms, total: 47 ms
Wall time: 11.4 s


In [29]:
div_stocks

,ticker,name,annual_div,div_yield,div_consistency_score
0,FCCY,1st Constitution Bancorp,0.36,2.8,0.918750
1,SRCE,1st Source,1.12,3.4,0.886364
2,MMM,3M Company,5.88,4.0,0.882809
3,ABBV,AbbVie,4.72,5.6,0.826935
4,ACCO,ACCO Brands,0.26,4.1,0.772959
...,...,...,...,...,...
740,WYND,Wyndham Destinations,1.85,7.3,0.000000
741,WH,Wyndham Hotels & Resorts,1.28,2.9,0.000000
742,XEL,Xcel Energy,1.72,2.9,0.000000
743,XRX,Xerox Holdings,1.00,5.5,0.000000


In [30]:
min_div = 4.0

approved_div_stocks = div_stocks[div_stocks['div_yield']>min_div]

approved_div_stocks

,ticker,name,annual_div,div_yield,div_consistency_score
1,SRCE,1st Source,1.12,3.4,0.886364
2,MMM,3M Company,5.88,4.0,0.882809
3,ABBV,AbbVie,4.72,5.6,0.826935
4,ACCO,ACCO Brands,0.26,4.1,0.772959
5,ADTN,ADTRAN,0.36,3.2,0.874991
...,...,...,...,...,...
738,WOR,Worthington Industries,0.96,3.6,0.000000
739,WPC,WP Carey,4.16,6.7,0.000000
740,WYND,Wyndham Destinations,1.85,7.3,0.000000
743,XRX,Xerox Holdings,1.00,5.5,0.000000


In [5]:
div_stocks['div_consistency_score'] = 0.0

In [6]:
def trim_div_data(stockObj):
    actns = stockObj.actions
    for i,r in actns.iterrows():
        if r['Dividends'] > 0:
            first_div_date = i
            break

    actns = actns.loc[first_div_date:]
    return actns, first_div_date

In [7]:
def add_div_data_details(actns, first_div_date):
    actns['days_since_first_div'] = (actns.index-first_div_date).days
    actns['year'] = actns.index.year
    
    h = []
    l = []
    for i,r in actns.iterrows():
        prices = stockObj.history(start=i,end = i+pd.DateOffset(1)).iloc[0]
        h.append(prices['High'])
        l.append(prices['Low'])

    actns['Stock_High'] = h
    actns['Stock_Low'] = l
    
    actns['day_div_yield'] = actns['Dividends']/actns['Stock_High']
    return actns

In [8]:
def get_ind_div_data(stockObj):
    df, first_div_date = trim_div_data(stockObj)
    return add_div_data_details(df, first_div_date)

In [9]:
def get_div_consistency_score(stockObj):
    
    df = stockObj.actions
    df['year'] = df.index.year
    df = df[df['Dividends']>0]
    df = df.drop(columns=['Stock Splits'])
    
    ann_df = df.groupby(['year']).agg(
        {'Dividends': np.sum}).rename(columns=
        {'Dividends':'dividends'})
    start_year = ann_df.index[0]
    end_year = ann_df.index[len(ann_df)-1]+1
    ann_df = ann_df.reindex(list(range(start_year,end_year)), fill_value=0)
    
    consistancy_coeff = [1/(2**(x+1)) for x in range(len(ann_df))][::-1]
    year_progress = datetime.now().timetuple().tm_yday/365
    consistancy_coeff[-1] = consistancy_coeff[-1]*year_progress
    
    ann_df['div_consistency_score'] = consistancy_coeff
    
    highest_div = ann_df.iloc[0]['dividends']
    
#     add another weight to the last cell where i get the percentage of the year we've been in and multiply it by that

    for i,r in ann_df[1:].iterrows():      
#         i need to think about a better way to do this because right now
#         it compares the wrong two pieces
        if r['dividends'] < ann_df.loc[i-1]['dividends']:
            mult = r['div_consistency_score']
            div_perc = r['dividends']/highest_div
            ann_df.at[i, 'div_consistency_score'] = mult * div_perc
        else:
            highest_div = r['dividends']
    
    return ann_df['div_consistency_score'].sum()

In [10]:
%%time
for index, row in approved_div_stocks[:15].iterrows():
    print(row['ticker'])
    stockObj = yf.Ticker(row['ticker'])
    div_consis = get_div_consistency_score(stockObj)
    print(div_consis)
    div_stocks.at[index, 'div_consistency_score'] = div_consis

FCCY
0.9187500000000001
SRCE
0.886363627057731
MMM
0.8828090109435487
ABBV
0.8269348714953271
ACCO
0.7729591836734694
ADTN
0.8749906788701597
ADES
0.5625
AES
0.875831022906072
AFL
0.8286469658199777
AGNC
0.6333040106233465
ADC
0.7596577824746
ALX
0.8711632498848121
ALE
0.8943697137851006
AB
0.8120776839471812
LNT
0.9002202004907858
CPU times: user 1.66 s, sys: 141 ms, total: 1.81 s
Wall time: 7.25 s


In [11]:
div_stocks[:15]

,ticker,name,annual_div,div_yield,div_consistency_score
0,FCCY,1st Constitution Bancorp,0.36,2.8,0.918750
1,SRCE,1st Source,1.12,3.4,0.886364
2,MMM,3M Company,5.88,4.0,0.882809
3,ABBV,AbbVie,4.72,5.6,0.826935
4,ACCO,ACCO Brands,0.26,4.1,0.772959
5,ADTN,ADTRAN,0.36,3.2,0.874991
6,ADES,Advanced Emissions Solutions,1.00,13.4,0.562500
7,AES,AES Corp,0.57,4.5,0.875831
8,AFL,AFLAC,1.12,3.1,0.828647
9,AGNC,AGNC Investment,1.44,11.3,0.633304


In [13]:
"""make sure i get the percent of the year and multiply the first coefficient by that percent for all """

'make sure i get the percent of the year and multiply the first coefficient by that percent for all '

In [25]:
x = [1/(2**(x+1)) for x in range(10)][::-1]

In [26]:
x

[0.0009765625,
 0.001953125,
 0.00390625,
 0.0078125,
 0.015625,
 0.03125,
 0.0625,
 0.125,
 0.25,
 0.5]

In [27]:
x[-1] = x[-1]*datetime.now().timetuple().tm_yday/365

In [28]:
x

[0.0009765625,
 0.001953125,
 0.00390625,
 0.0078125,
 0.015625,
 0.03125,
 0.0625,
 0.125,
 0.25,
 0.3273972602739726]